In [5]:
import numpy as np

# models
from scipy.optimize import curve_fit # multip. linear regression
from sklearn.svm import SVR # support vector forrest
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import time

# random data, 3d
np.random.seed(42)
x_data = np.random.uniform(0, 10, 100)
y_data = np.random.uniform(0, 10, 100)

results = []

# True model: z = a*x + b*y + c + noise
a_true, b_true, c_true = 2.5, -1.3, 4.7
noise = np.random.normal(0, 0.5, 100)
z_data = a_true * x_data + b_true * y_data + c_true + noise

# Feature matrix
X = np.column_stack((x_data, y_data))

# Split the data into training and testing sets
X_train, X_test, z_train, z_test = train_test_split(X, z_data, test_size=0.2, random_state=42)

# 1. Multilinear Regression using curve_fit
def linear_model(X, a, b, c):
    x, y = X  # X is a tuple (x, y)
    return a * x + b * y + c

# Fit the model using curve_fit
initial_guess = [1, 1, 1]  # Initial guess for a, b, c
start_time = time.time()
params, _ = curve_fit(linear_model, (x_data, y_data), z_data, p0=initial_guess)
a_fit, b_fit, c_fit = params

# Predict
z_pred_curve_fit = linear_model((x_data, y_data), a_fit, b_fit, c_fit)

# Calculate metrics
time_curve_fit = time.time() - start_time
mse_curve_fit = mean_squared_error(z_data, z_pred_curve_fit)
r2_curve_fit = r2_score(z_data, z_pred_curve_fit)

# 2. Support Vector Regression (SVR)
start_time = time.time()
svr = SVR(kernel='rbf', C=10, epsilon=0.1)
svr.fit(X_train, z_train)

# Predict
z_pred_svr = svr.predict(X)

# Calculate metrics
time_svr = time.time() - start_time
mse_svr = mean_squared_error(z_data, z_pred_svr)
r2_svr = r2_score(z_data, z_pred_svr)

# 3. Random Forest Regression
start_time = time.time()
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, z_train)

# Predict
z_pred_rf = random_forest.predict(X)

# Calculate metrics
time_rf = time.time() - start_time
mse_rf = mean_squared_error(z_data, z_pred_rf)
r2_rf = r2_score(z_data, z_pred_rf)

# 4. Artificial Neural Network (ANN) Regression
start_time = time.time()
ann = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42, solver='adam')
ann.fit(X_train, z_train)

# Predict
z_pred_ann = ann.predict(X)

# Calculate metrics
time_ann = time.time() - start_time
mse_ann = mean_squared_error(z_data, z_pred_ann)
r2_ann = r2_score(z_data, z_pred_ann)


results.append(["Multilinear Regression (curve_fit)", mse_curve_fit, r2_curve_fit, time_curve_fit])
results.append(["Support Vector Regression (SVR)", mse_svr, r2_svr, time_svr])
results.append(["Random Forest Regression", mse_rf, r2_rf, time_rf])
results.append(["Artificial Neural Network (ANN)", mse_ann, r2_ann, time_ann])

results_df = pd.DataFrame(results, columns=["Model", "MSE", "R-squared", "Time Taken (s)"])
print(results_df)

                                Model       MSE  R-squared  Time Taken (s)
0  Multilinear Regression (curve_fit)  0.236418   0.996607        0.001005
1     Support Vector Regression (SVR)  0.310989   0.995537        0.004992
2            Random Forest Regression  0.600882   0.991377        0.219993
3     Artificial Neural Network (ANN)  0.257498   0.996305        0.344006
